In [1]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from bisect import bisect_left

In [2]:
path_to_folder = '/Users/rwkoops/PycharmProjects/IR_project/IR_project_02/'
prefixes = pd.read_pickle(path_to_folder + 'created_sample/prefixes_1m.pickle')

samples_hist = pd.read_pickle(path_to_folder + 'created_sample/sample_hist_1m.pickle')   
samples_hist['Query_clean'] = samples_hist['Query'].map(lambda string: re.sub(r"[^A-Za-z0-9]+", " ", str(string)).lower().strip())
samples_hist['Query_clean'].replace('', np.nan, inplace=True)
samples_hist.dropna(subset=['Query_clean'], inplace=True)
hist_q = samples_hist['Query_clean']

In [3]:
hist_dict = Counter(list(samples_hist['Query_clean']))
hist_list = hist_dict.most_common()
hist_sort = sorted([i[0] for i in hist_list])

prefixes = prefixes.sort_values(by=['Prefix'])
prefixes.columns = ['Query', 'Qid', 'Prefix']
prefix_q = prefixes[['Query','Prefix']]

In [4]:
# Check if it contains suffix
def bisect_contains_check(hist_list, prefix):
    try:
        return hist_list[bisect_left(hist_list, prefix)].startswith(prefix)
    except IndexError:
        return False

# Returns the prefix keys
def bisect_list_slice(hist_list, prefix):
    return hist_list[bisect_left(hist_list, prefix):
         bisect_left(hist_list, prefix[:-1] + chr(ord(prefix[-1])+1))]

In [5]:
SumRR = 0
i = 0

for pr_q in prefixes.itertuples():
    
    if bisect_contains_check(hist_sort, pr_q.Prefix):
        hist_match_list = bisect_list_slice(hist_sort, pr_q.Prefix)
        
        temp_list = []
        
        for key in hist_match_list:
            if (hist_dict.get(key) is None):
                temp_list.append([key, 0])
            else:
                temp_list.append([key, hist_dict.get(key)])
    
        temp_list.sort(key=lambda x:str(x[1]),reverse=True)
        top_8 = [i[0] for i in temp_list[:8]]
        
        index = len(top_8) + 1
        
        for i in top_8:
            index -= 1
            if str(i) == str(pr_q.Query):
                SumRR += index/(len(top_8))
                break;

MRR = SumRR / (prefixes.shape)[0]
print(MRR)

0.21536900117884888
